<a href="https://colab.research.google.com/github/srinisuryadevara/Course/blob/master/Recomendation_Engine1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# importing surprise for collaborative filtering model
! pip install surprise

# import necessary libraries
import numpy as np
import pandas as pd
from surprise import NMF
from surprise import SVD
import matplotlib.pyplot as plt
from surprise import Reader, Dataset, KNNBasic
from sklearn.metrics import mean_squared_error
from surprise.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
 #reading rating, movies and user dataset

# rating dataset
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('drive/MyDrive/amazing_recommenderr/u.data',  sep='\t', names=r_cols,
encoding='latin-1')

# movies dataset
i_cols = ['movie_id', 'title' ,'release date','video release date', 'IMDb URL',
          'unknown', 'Action', 'Adventure','Animation', 'Children\'s', 'Comedy', 'Crime',
          'Documentary', 'Drama', 'Fantasy','Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('drive/MyDrive/amazing_recommenderr/u.item',  sep='|', names=i_cols, encoding='latin-1')

# user dataset
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('drive/MyDrive/amazing_recommenderr/u.user', sep='|', names=u_cols,
encoding='latin-1')
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [10]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [11]:
movies.head()

,movie_id,title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [12]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [13]:
# Define a Reader object

# The Reader object helps in parsing the file or dataframe containing ratings
ratings = ratings.drop(columns='timestamp')
reader = Reader()

# Dataset creation
data = Dataset.load_from_df(ratings, reader)

In [14]:
# Models

# 1. KNN model
knn = KNNBasic()

# 2.SVD Model
svd = SVD()

# 3. Non-negative Matrix Factorization (NMF) model:
# The utility matrix is factorized into two matrices, just like SVD.
# However, under NMF, each matrix can only have non-negative values.
nmf = NMF()

In [15]:
#Evaluating the performance of KNN in terms of RMSE and MAE of KNN
cross_validate(knn, data, measures=['RMSE', 'mae'], cv = 3)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.98981928, 0.98452801, 0.99441514]),
 'test_mae': array([0.78166284, 0.77870386, 0.78608579]),
 'fit_time': (0.6111681461334229, 0.22591090202331543, 0.14020848274230957),
 'test_time': (4.323861122131348, 3.5283122062683105, 3.3731801509857178)}

In [16]:
#Evaluate the performance of SVD in terms of RMSE and MAE of SVD
cross_validate(svd, data, measures=['RMSE', 'mae'], cv = 3)

{'test_rmse': array([0.95063671, 0.9430833 , 0.94186471]),
 'test_mae': array([0.74932511, 0.74399945, 0.74423348]),
 'fit_time': (1.046217918395996, 0.833397626876831, 1.0740954875946045),
 'test_time': (0.15097784996032715, 0.3133728504180908, 0.2460768222808838)}

In [17]:
#Evaluate the performance of NMF in terms of RMSE and MAE of SVD
cross_validate(nmf, data, measures=['RMSE', 'mae'], cv = 3)

{'test_rmse': array([0.96827686, 0.97526241, 0.97402008]),
 'test_mae': array([0.76065929, 0.76648532, 0.76556318]),
 'fit_time': (1.4835941791534424, 1.9079127311706543, 1.493959665298462),
 'test_time': (0.14066624641418457, 0.32166624069213867, 0.13866090774536133)}

In [18]:
# installing required packages
!pip install lightfm
!pip install scrapbook
!pip install recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=867232 sha256=639e104ae339656e5d8543c2d27205e70583c696da08ac838bf5caa2317c5d6c
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.3 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 0.6.0 Requires-Python >=3.6, <=3.8; 0.7.0 Requires-Python >=3.6, <3.8; 1.0.0 Requires-Python >=3.6, <3.9; 1.1.0 Requires-Python >=3.6, <3.10; 1.1.1 Requires-Python >=3.6, <3.10
ERROR: Could not find a version that satisfies the requirement recommenders (from versions: none)
ERROR: No matching distribution found for recommenders
